# Sensex 30 - Direct Indexing

## Installing the dependencies
There are a number of requirements for this project. I'll keep adding them as I go along
1. lxml
2. bs4
3. selenium
4. pandas
5. numpy
6. openpyxm

In [1]:
import lxml
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import time
import math

## Initialising Selenium Webdriver
First and foremost, we need the names of the companies in the Sensex 30. I scrape the names from <a href="https://www.bseindia.com/sensex/code/16">BSE's Website</a>. It gets updated in real-time. 

Second, we also need the real-time stock price and market cap of the companies in Sensex 30. I scrape the data from Google. I looked for easy to use free APIs for this, but I couldn't find any. Since, this is an on-book fund, I am not using the APIs of any brokers, like Zerodha. In the future, if the on-book fund, gives me adequate returns, I will move to getting the data from brokers and executing the trades through the API too.

In [2]:
opts = Options()
opts.headless = True
chrome_driver = "./chromedriver"
driver = webdriver.Chrome(executable_path=chrome_driver, options=opts)

## Scraping BSE's Website
I am using Selenium to get the HTML code and then parse it using BeautifulSoup. I tried two libraries before this:
1. requests - There is dynamic content on BSE's Website so the requests module was not working
2. requests-html - This module has a function which can render the website and then get the HTML code, for some reason, .render() function was giving me errors

So, to make the code as safe as possible, I decided to go with Selenium

In [3]:
driver.get("https://www.bseindia.com/sensex/code/16")
soup = BeautifulSoup(driver.page_source, "lxml")

I am using 2 try-catch blocks in the code below. This is because, '\n', i.e. new line, is also part a part of the list which is returned using .contents. 

I will change this try-catch block to something else that works, but this works for now.

In [4]:
sensex_list = []
mainDiv_contents = soup.find(id="mainDiv").contents[0].contents[0]
for tr in mainDiv_contents:
    try:
        for td in tr.contents:
            try:
                stock = td.contents[0].contents[0].contents[0].contents[0].string.strip()
                sensex_list.append(stock)
            except AttributeError:
                continue
    except AttributeError:
        continue

sensex_list = sensex_list[:30]
sensex_list[0:30:5]  # A glimpse of the list

['TITAN', 'MARUTI', 'LT', 'ASIANPAINT', 'NESTLEIND', 'DRREDDY']

## Creating a Pandas DataFrame which will store all the Data
This dataframe will store today's updated data. We will use this dataframe to perform the necessary calculations for the day and eventually place our orders.

In [5]:
column_list = ['Name', 'Ticker Symbol', 'Price (in Rs)', 'FF Market Cap (in Cr)', '% by Market Cap']
data = pd.DataFrame(columns=column_list)

## Scraping for Today's Price & Free Floating Market Cap
I looked into many API's including but not restricted to Yahoo Finance, Alpha Vantage, Quandl. They were either paid or they were extremely complicated. I wanted the live quotes of each share. 

I had earlier decided to scrape the data from Google Finance, but there is a fundamental problem with this. All indices, Sensex included, give the weightage to each company on the basis of their "free floating market cap". This is the market cap calculated on the number of shares available to the public to trade. Free Floating Market Cap drastically changes the weightage of each stock.

I had to go through a cumbersome process to scrape this data. I first go to <a href="www.screener.in">Screener</a> and search for individual stocks in the "sensex_list". I searched for the individual stocks on Screener because, this was a very reliable search and I almost always got the correct stock. From the Screener page, I get the link to that company's BSE page. I then go to the company's BSE page to scrape its:
1. Name
2. Ticker Symbol
3. Current Price
4. Free Floating Market Cap

In [6]:
for stock in sensex_list:
    if stock == "INFY*":
        stock = "INFY"
    driver.get(f"https://www.screener.in/company/{stock}/consolidated/")
    time.sleep(1)
    bse_website_link = driver.find_element_by_xpath('//*[@id="top"]/div[2]/a[2]').get_attribute('href')

    driver.get(bse_website_link)
    while True:
        try:
            name = driver.find_element_by_xpath(
                '//*[@id="getquoteheader"]/div[6]/div/div[3]/div/div[1]/div[1]/div[1]/div[2]/div/h1').text.upper()
            ticker_symbol = driver.find_element_by_xpath(
                '//*[@id="getquoteheader"]/div[6]/div/div[3]/div/div[1]/div[1]/div[1]/div[2]/div/div[2]')
            ticker_symbol = ticker_symbol.text.split("|")[0].strip()[1:]
            price = float(driver.find_element_by_xpath('//*[@id="idcrval"]').text)
            ff_market_cap = driver.find_element_by_xpath(
                '//*[@id="getquoteheader"]/div[6]/div/div[4]/div/div[3]/div/table/tbody/tr[7]/td[2]')
            ff_market_cap = ff_market_cap.text  # In Crores
#           ff_market_cap = float(ff_market_cap.replace(",","")) * 1000000000
            ff_market_cap = float(ff_market_cap.replace(",",""))
            break
        except:
            time.sleep(0.25)
            
    data = data.append(
        pd.Series(
            [
                name,
                ticker_symbol,
                price,
                ff_market_cap,
                np.nan,
            ],
            index=column_list
        ),
        ignore_index=True,
    )
    
    print(name)

TITAN COMPANY LIMITED
MAHINDRA & MAHINDRA LTD.
RELIANCE INDUSTRIES LTD.
AXIS BANK LTD.
TATA CONSULTANCY SERVICES LTD.
MARUTI SUZUKI INDIA LTD.
INFOSYS LTD.
ITC LTD.
HCL TECHNOLOGIES LTD.
HOUSING DEVELOPMENT FINANCE CORP.LTD.
LARSEN & TOUBRO LTD.
STATE BANK OF INDIA
INDUSIND BANK LTD.
KOTAK MAHINDRA BANK LTD.
BHARTI AIRTEL LTD.
ASIAN PAINTS LTD.
BAJAJ AUTO LTD.
ULTRATECH CEMENT LTD.
TECH MAHINDRA LTD.
SUN PHARMACEUTICAL INDUSTRIES LTD.
NESTLE INDIA LTD.
HINDUSTAN UNILEVER LTD.
POWER GRID CORPORATION OF INDIA LTD.
ICICI BANK LTD.
HDFC BANK LTD
DR.REDDY'S LABORATORIES LTD.
NTPC LTD.
BAJAJ FINANCE LIMITED
BAJAJ FINSERV LTD.
TATA STEEL LTD.


In [7]:
data.head()

,Name,Ticker Symbol,Price (in Rs),FF Market Cap (in Cr),% by Market Cap
0,TITAN COMPANY LIMITED,TITAN,1770.25,73865.36,NaN
1,MAHINDRA & MAHINDRA LTD.,M&M,757.85,72545.82,NaN
2,RELIANCE INDUSTRIES LTD.,RELIANCE,2071.50,669739.98,NaN
3,AXIS BANK LTD.,AXISBANK,720.95,183433.33,NaN
4,TATA CONSULTANCY SERVICES LTD.,TCS,3217.90,333288.97,NaN


## Calculating and Assigning the Weightage as of Today
I took the data in the "FF Market Cap" column and I copied it into a series. I then take the sum of this series, which represents the total market capitalising of all the stocks in Sensex 30 in Crores. Now, using this sum, I calculate the Weightage of the individual stock and sort it from highest to lowest weightage.

Weightage = (Market Cap of Company) / (Market Cap of Index)


In [8]:
total_market_cap = data["FF Market Cap (in Cr)"].sum()
for i in range(30):
    market_weight_percent = round(((data["FF Market Cap (in Cr)"][i]/total_market_cap) * 100), 2)
    data.loc[i, "% by Market Cap"] = market_weight_percent
data.sort_values(by="% by Market Cap", ascending=False, inplace=True, ignore_index=True)
data.head()

,Name,Ticker Symbol,Price (in Rs),FF Market Cap (in Cr),% by Market Cap
0,RELIANCE INDUSTRIES LTD.,RELIANCE,2071.50,669739.98,11.35
1,HDFC BANK LTD,HDFCBANK,1422.75,621795.01,10.54
2,INFOSYS LTD.,INFY*,1631.35,604481.18,10.25
3,ICICI BANK LTD.,ICICIBANK,681.55,472052.95,8.00
4,HOUSING DEVELOPMENT FINANCE CORP.LTD.,HDFC,2462.30,444629.74,7.54


## Comparing Weightage between the Master File and Today's Data
This is the "tracking" portion of our Direct Indexing. I have a master file which gets updated everyday. This file contains the following data:
1. Name
2. Ticker Symbol
3. Price (in Rs)
4. FF Market Cap (in Cr)
5. % by Market Cap
6. Buy Price
7. Quantity
8. Date of Buy

We will compare the "% by Market Cap" Column in master_df and in data, to look for changes in the weightage of companies in the index. We will flag changes of +/- 0.85% in the weightage. 
<ul>
    <li>For +0.85% in weightage -> We will re-arrange our portfolio by buying more shares of the company</li>
    <li>For -0.85% in weightage -> We will re-arrange our portfolio by selling shares of the company</li>
</ul>

There are a few things I am not sure of yet:
<ol>
    <li>
        I will have to round off and calculate a lot of the portfolio allocation. A lot of the cash will remain becuase of the rounding down, so will have to keep track of that too.
    </li>
    <li>
        Lets say, Reliance moved from 11.00% to 11.85%, how do I buy this 0.85% in weightage
        <ul>
        <li>
            I have already spent all the cash I had in buying the first lot of shares
        </li>
        <li>
            One way to think about this is that on the day of +0.85% increase, I rebalance my portfolio. That is, I buy and sell for each minute difference in the weightage, because the total must come up to 100. 
        </li>
        <li>
            Now, for the above to be done, I will have to re-calculate to get atleast 1 share of each company. With that one share, now calculate the number of shares each company should have. Using that as the reference, I first sell all the extra shares and then with the cash I get, check if I can afford to buy the extra shares added due to +0.85% increase in weightage.
        </li>
        <li>
            I think that 0.85% is a big change in weightage and it is tough to realise such extreme changes in market caps. So, probably I should change 0.85% to 0.5% or lower. 
        </li>
        <li>
            A similar concept is to be applied for a -0.85% change. Because, if the weightage of one company is decreasing, the weightage of some other company must increase.
        </li>
        </ul>
    </li>
</ol>

In [9]:
with open('Master DataFrame.csv', 'r') as mdf_csv:
    master_df = pd.read_csv(mdf_csv)
master_df.head()

,Name,Ticker Symbol,Price (in Rs),FF Market Cap (in Cr),% by Market Cap,Buy Price,Quantity,Date of Buy
0,RELIANCE INDUSTRIES LTD.,RELIANCE,2035.40,658068.43,11.23,NaN,NaN,NaN
1,HDFC BANK LTD,HDFCBANK,1425.80,623127.97,10.63,NaN,NaN,NaN
2,INFOSYS LTD.,INFY*,1610.25,596662.78,10.18,NaN,NaN,NaN
3,ICICI BANK LTD.,ICICIBANK,682.70,472849.46,8.07,NaN,NaN,NaN
4,HOUSING DEVELOPMENT FINANCE CORP.LTD.,HDFC,2440.75,440738.35,7.52,NaN,NaN,NaN


In [10]:
# This compares the master and today's data.
# It raises a flag for an absolute change of greater or equal to 0.5 Percentage Points
flag = False
for i in range(30):
    weight_master = master_df["% by Market Cap"][i]
    weight_data = data["% by Market Cap"][i]
    if abs(weight_master - weight_data) >= 0.5:
        flag = True
        break

flag

False

In [11]:
# Check if flag is true to continue with the rebalancing
# This is the rebalancing portion of the Project

if flag is not True:
    # Do other actions, buy/sell is not required
    quit()


## Comparing and Rebalancing Concept

Now that flag is true, we will buy or sell stocks. We start with assigning what should be the quantity we hold today. I put that data in the data DataFrame itself in a new column called quantity

#### Note: While comparing master and data, we cannot use index. Because let's say in the master Reliance is at first position and HDFC Bank is at 2nd position, but today the reliance falls to 2nd position. Then we are comparing apples to oranges

Rebalancing our Portfolio:

We compare the quantity of each share in data and in the master DataFrame. There will be 3 variables, one which calculates the amount we will have to spend, that is the extra number of shares we will have to buy. The second variable will keep track of the amount we will receive that is in profit, i.e, the shares which we will sell at a price greater than our cost price. The third variable will store the amount we will receive which is in loss, i.e, the shares which we will sell at a price lower than our cost price. Using these three variables, plus our available cash, we will first determine if any transaction is possible at all. This means that, if the total cash is in -ve then we cannot rebalance the entire portfolio. If we cannot rebalance our portfolio, there can be a feature added which will selectively rebalance our portfolio but that is something we will look into later. Right now, if the total cash comes out to be -ve, we will skip the rebalancing and let the portfolio stay as it is.

Now, if the total cash is greater or equal to zero. This means that we have just enough money to rebalance our portfolio. At this point, we will actually (on paper that is) start buying and selling stocks. We will first have to sell off our stocks to raise enough capital to buy the required securities. I think, in the above step, we can maintain a dictionary, with the key being the index number and the value being the amount. Then, using the index numbers, we can loop through the sales, updating the cash with each sale. Finally, afte the cash is updated, we will start looping through the buy section. 

This is the final portfolio rebalancing. There are a number of variables and concepts involved. I think it is best to draw a flowchart to understand the rebalancing part well and selecting the variables and techniques correctly.

In [1]:
# Now that flag is true, we will buy or sell stocks
# We start with assigning what should be the quantity we hold today
# I put that data in the data DataFrame itself in a new column called quantity

cash = 2500000
cash_required = 0

data["Quantity"] = 0
for i in range(30):
    weight = data.loc[i, "% by Market Cap"]
    price = data.loc[i, "Price (in Rs)"]
    quant = math.floor((((weight/100)*cash)/price))
    cash_required += quant * price

NameError: name 'data' is not defined

In [2]:
# In the above box, we calculate how much money is required to buy all the shares in the portfolio
# I will do a similar calculation for the master too.
# I will sell the required shares, 

if cash_required > cash:
    # The cash required to buy the required shares is less than that which is available
    print("Cash is Less -> Program has broken, Check")
    quit()

cash_required = 0
for i in range(30):
    weight = today_df.loc[i, "% by Market Cap"]
    price = today_df.loc[i, "Price (in Rs)"]
    quant = math.floor((((weight/100)*cash)/price))
    today_df.loc[i, "Quantity"] = quant
    cash_required += (price * quant)

cash -= cash_required

print(cash)
today_df

NameError: name 'data' is not defined

In [14]:
cash_test = 2500000
cash_required = 0
for i in range(30):
    weight = master_df.loc[i, "% by Market Cap"]
    price = master_df.loc[i, "Price (in Rs)"]
    quant = math.floor((((weight/100)*cash_test)/price))
    master_df.loc[i, "Quantity"] = quant
    master_df.loc[i, "Buy Price"] = price
    cash_required += (price * quant)
    
cash_test -= cash_required

print(cash)
master_df

48878.15000000037


,Name,Ticker Symbol,Price (in Rs),FF Market Cap (in Cr),% by Market Cap,Buy Price,Quantity,Date of Buy
0,RELIANCE INDUSTRIES LTD.,RELIANCE,2035.40,658068.43,11.23,2035.40,137.0,NaN
1,HDFC BANK LTD,HDFCBANK,1425.80,623127.97,10.63,1425.80,186.0,NaN
2,INFOSYS LTD.,INFY*,1610.25,596662.78,10.18,1610.25,158.0,NaN
3,ICICI BANK LTD.,ICICIBANK,682.70,472849.46,8.07,682.70,295.0,NaN
4,HOUSING DEVELOPMENT FINANCE CORP.LTD.,HDFC,2440.75,440738.35,7.52,2440.75,77.0,NaN
5,TATA CONSULTANCY SERVICES LTD.,TCS,3167.50,328068.87,5.60,3167.50,44.0,NaN
6,KOTAK MAHINDRA BANK LTD.,KOTAKBANK,1654.95,242810.97,4.14,1654.95,62.0,NaN
7,HINDUSTAN UNILEVER LTD.,HINDUNILVR,2334.05,208394.20,3.56,2334.05,38.0,NaN
8,LARSEN & TOUBRO LTD.,LT,1601.40,193442.75,3.30,1601.40,51.0,NaN
9,AXIS BANK LTD.,AXISBANK,709.00,180392.86,3.08,709.00,108.0,NaN
